In [13]:
from azureml.core import Workspace

src_folder = 'steps'
cluster_name = 'cpu-cluster'
env_name = 'data-drift-env'

DATASTORE_NAME = 'workspaceblobstore'
FILE_DATASET_NAME = 'datadrift_file_results'
json_file_path = f'datadrift/metrics/**/output_*.json'


ws = Workspace.from_config()
print('Ready to work with', ws.name)

Ready to work with evolve-ml


In [15]:
%%writefile -a $src_folder/utils.py

from contextlib import contextmanager
import os
import shutil


@contextmanager
def temp_directory(dir_name = 'temp', **kwds):
    
    os.makedirs(dir_name, exist_ok=True)
    
    try:
        # print(client.server_info())

        yield dir_name

    except Exception:
        print(f"Unable to create '{dir_name}'")

    finally:
        shutil.rmtree(dir_name)

Writing steps/utils.py


In [10]:
%%writefile $src_folder/collect-data-drift-output.py

import argparse
from azureml.core import Workspace, Dataset, Datastore, Run
from azureml.core.run import _OfflineRun
from azureml.data.dataset_factory import DataType



def parse_args():
    parser = argparse.ArgumentParser()
    #parser.add_argument('--output', dest='output', required=True)
    #parser.add_argument('--datadir', dest='datadir', required=True)

    return parser.parse_args()


args = parse_args()
print(f'Arguments: {args.__dict__}')


run = Run.get_context()
ws = Workspace.from_config() if type(run) == _OfflineRun else run.experiment.workspace

# Crate FileDataSet based on datadrift metrics which are saved in datastore as json files
dstore = Datastore.get(ws, DATASTORE_NAME)
file_dataset = Dataset.File.from_files(path=(dstore,json_file_path))
file_dataset.register(ws, FILE_DATASET_NAME, create_new_version=True)

#TODO: 
## add filter dataset
## add arguments instead of constants


Overwriting steps/collect-data-drift-output.py


In [24]:
%%writefile $src_folder/transform-data-drift-output.py

import argparse
import json
import bigjson
import os
import utils
from azureml.core import Workspace, Dataset, Datastore, Run
from azureml.core.run import _OfflineRun

TEMP_DIRECTORY = 'temp'

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
    parser.add_argument('--transformed-data', type=str, dest='transformed_data', default='transformed_data', help='Folder for results')
    return parser.parse_args()

# Get parameters
args = parse_args()
print(f'Arguments: {args.__dict__}')

save_folder = args.transformed_data
os.makedirs(save_folder, exist_ok=True)

# Get the experiment run context
run = Run.get_context()
file_dataset = run.input_datasets['raw_data']
ws = Workspace.from_config() if type(run) == _OfflineRun else run.experiment.workspace

with utils.temp_directory(TEMP_DIRECTORY):
    # Download json files defined by the dataset to temp directory
    json_file_paths = file_dataset.download(f'{TEMP_DIRECTORY}', overwrite=True)

    # Convert json files to jsonl files (in local directory) 
    for json_path in json_file_paths:
        
        # Read json file in streaming mode
        with open(json_path, 'rb') as f:
            json_data = bigjson.load(f)
            # Replace file name extension
            jsonl_path = os.path.splitext(json_path)[0]+'.jsonl'

            # Open jsonl file  
            with open(jsonl_path, 'w') as jsonl_file:
                # Iterates over input json
                for data in json_data:
                    # Converts json to a Python dict  
                    dict_data = data.to_python()
                    
                    # Saves the data to jsonl file
                    jsonl_file.write(json.dumps(dict_data)+"\n")
                    
        # Delete json file
        os.remove(json_path)

    # Upload jsonl files to datastore
    print("Saving Transformed Data...")
    output_dataset = Dataset.File.upload_directory(f'{TEMP_DIRECTORY}', target=save_folder)

#TODO: 
## move to util


Overwriting steps/transform-data-drift-output.py


In [17]:
%%writefile $src_folder/save-data-drift-output.py

import argparse
from azureml.core import Dataset, Datastore, Run
from azureml.core.run import _OfflineRun
from azureml.data.dataset_factory import DataType

PARTITION_FORMAT = '{DATADRIFT_ID}/{PARTITION_DATE:yyyy/MM/dd}/output_{RUN_ID}.json'


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--transformed-data", type=str, dest='transformed_data', help='transformed data')
    return parser.parse_args()

args = parse_args()
print(f'Arguments: {args.__dict__}')
transformed_data = args.transformed_data


run = Run.get_context()
ws = Workspace.from_config() if type(run) == _OfflineRun else run.experiment.workspace

# Crate TabularDataSet based on converted jsonl files
output_dataset = Dataset.Tabular.from_json_lines_files(path=transformed_data, partition_format=PARTITION_FORMAT)
output_dataset = output_dataset.register_on_complete(name='datadrift_results_pipeline', description = 'datadrift results pipeline')


Overwriting steps/save-data-drift-output.py


In [2]:
if False:
    from azureml.core.environment import Environment
    from azureml.core.conda_dependencies import CondaDependencies

    myenv = Environment(name=env_name)
    conda_dep = CondaDependencies()

    conda_dep.add_pip_package('bigjson')
    conda_dep.add_pip_package('azureml-defaults')

    # Adds dependencies to PythonSection of myenv
    myenv.python.conda_dependencies=conda_dep

    myenv.register(ws).build(ws)

In [18]:
from azureml.core import RunConfiguration, ComputeTarget, Environment

run_config = RunConfiguration()
run_config.environment = Environment.get(ws, env_name)
compute_target = ComputeTarget(ws, cluster_name)


In [22]:
from azureml.pipeline.core import PipelineData, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import ComputeTarget, Datastore, Dataset
from azureml.data import OutputFileDatasetConfig

#datadir_param = PipelineData('datadir', is_directory=True)
#evolve_param = PipelineData('evolve')
#azure_param = PipelineData('azure')
#compare_param = PipelineData('compare')

#collection_param = PipelineParameter(name="collection", default_value='test_datasets')
#repo_param = PipelineParameter(name="repo", default_value='test')


# Get data-drift output dataset
dstore = Datastore.get(ws, DATASTORE_NAME)
metrics_ds = Dataset.File.from_files(path=(dstore,json_file_path)) # add filter dataset

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
transformed_data = OutputFileDatasetConfig("transformed_data")

# Step 1, Run the data transofrm script (from json to jsonl)
transform_step = PythonScriptStep(
    name='transform data drift output',
    source_directory=src_folder,
    script_name='transform-data-drift-output.py',
    arguments = ['--input-data', metrics_ds.as_named_input('raw_data'),
                '--transformed-data', transformed_data],
    compute_target=compute_target, 
    runconfig=run_config, 
    allow_reuse=False,    
)

# Step 2, Run the saving transformed data script
save_step = PythonScriptStep(
    name='save data drift output',
    source_directory=src_folder,
    script_name='save-data-drift-output.py',
    arguments = ['--transformed-data', transformed_data.as_input()],
    compute_target=compute_target, 
    runconfig=run_config,    
    allow_reuse=False,
)

print("Pipeline steps defined")

Pipeline steps defined


In [25]:
from azureml.core import Experiment
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [transform_step, save_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'data-drift-output-exeriment')
pipeline_run = experiment.submit(pipeline) 
print("Pipeline submitted for execution.")

RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)


Pipeline is built.
Created step transform data drift output [3314587d][1a2aeb95-ff87-40d5-8a68-f0b94f253ae6], (This step will run and generate new outputs)
Created step save data drift output [62a1a57f][837600f9-393a-424e-b86f-e952ede0fad1], (This step will run and generate new outputs)
Submitted PipelineRun cec01287-d7a8-4314-80c4-819a32b3a19b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/cec01287-d7a8-4314-80c4-819a32b3a19b?wsid=/subscriptions/06c3d5bb-46a2-4d92-9508-c018d06f6452/resourcegroups/evolve-team-rg/workspaces/evolve-ml&tid=72a43063-967e-43c8-8121-0823266b2701
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: cec01287-d7a8-4314-80c4-819a32b3a19b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/cec01287-d7a8-4314-80c4-819a32b3a19b?wsid=/subscriptions/06c3d5bb-46a2-4d92-9508-c018d06f6452/resourcegroups/evolve-team-rg/workspaces/evolve-ml&tid=72a43063-967e-43c8-8121-0823266b2701
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 56d7a1a3-4b38-4b47-9caf-c0b4394f2472
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/56d7a1a3-4b38-4b47-9caf-c0b4394f2472?wsid=/subscriptions/06c3d5bb-46a2-4d92-9508-c018d06f6452/resourcegroups/evolve-team-rg/workspaces/evolve-ml&tid=72a43063-967e-43c8-8121-0823266b2701
StepRun( transform data drift output ) Status: Running

StepRun(transform data drift output) Execution Summary
StepRun( transform data drift output ) Status: Finished
{'runId': '56d7a1a3-4b38-4b47-9caf-c0b4394f2472', 'target': 'cpu-cluster', 'status': 'Completed', 'startTimeUtc': '2022-04-16T20:19:36.234675Z', 'endTimeUtc': '2022-04-16

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{'code': data-capability.DatasetMountSession:input_23a007d8.ExecutionError, 'message': \nError Code: ScriptExecution.StreamAccess.NotFound\n, 'target': , 'category': UserError, 'error_details': [{'key': NonCompliantReason, 'value': \nError Code: ScriptExecution.StreamAccess.NotFound\nFailed Step: c24a2833-76a6-45e6-adf2-90fcdaac1dad\nError Message: ScriptExecutionException was caused by StreamAccessException.\n  StreamAccessException was caused by NotFoundException.\n    Found no resources for the input provided: 'https://evolveml3040211544.blob.core.windows.net/azureml-blobstore-899eb723-9471-475e-9ea2-abcad8a41de5/dataset/56d7a1a3-4b38-4b47-9caf-c0b4394f2472/transformed_data/'\n| session_id=9a8b8bd8-f880-45ef-9043-603a35b7028b}, {'key': StackTrace, 'value':   File \"/opt/miniconda/envs/data-capability/lib/python3.7/site-packages/data_capabilit

In [ ]:
from azureml.pipeline.core import PipelineRun
from azureml.core import Experiment

# Publish the pipeline from the run
submitted_pipeline_run = PipelineRun(experiment=Experiment(experiment, run_id=pipeline_run.id))
published_pipeline = submitted_pipeline_run.publish_pipeline(name='data-drift-output-pipeline',
    description='collect, transform and save datadrift output into dataset',
    version='1.0',
    continue_on_step_failure=False)

print('Pipeline scheduled.')

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Schedules a daily run of a published pipeline
daily = ScheduleRecurrence(frequency='Day', interval=1)
pipeline_schedule = Schedule.create(ws, name='data_drift_output_schedule',
                                        description='update data drift output every day',
                                        pipeline_id=published_pipeline.id,
                                        experiment_name='schedule_data_drift_output_pipeline',
                                        recurrence=daily)

print('Pipeline scheduled.')